<a href="https://colab.research.google.com/github/DataSavvyYT/AI-engineering-course/blob/main/05_agentic_ai/04_openai_swarm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q git+https://github.com/openai/swarm.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.9/160.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.4/226.4 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.8/358.8 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 90.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 115.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 28.2 MB/s eta 0:00:00


In [2]:
from swarm import Swarm, Agent

In [3]:
from google.colab import userdata
import os

# Ensure the API key is available
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [4]:
# Initialize Swarm client
client = Swarm()

In [5]:
# --- Define Tools ---

def process_refund(item_id):
    """Call this to process a refund for a specific item ID."""
    print(f"\n[MOCK BANKING SYSTEM] Processing refund for item {item_id}...")
    return f"Success! Refund processed for item {item_id}."

In [11]:
# --- Define Agents ---

# 1. Define the Specialist Agent first (Refund Agent)
refund_agent = Agent(
    name="Refund Agent",
    model="gpt-4o-mini",
    instructions="You are a refund specialist. efficiently process refunds. You have a tool 'process_refund' that you must use.",
    functions=[process_refund]
)

In [12]:
# 2. Define the Handoff Function
# This function returns the AGENT object itself. This triggers the handoff.
def transfer_to_refund_agent():
    """Call this function if the user asks about refunds, money back, or returns."""
    print("\n[HANDOFF] Transferring user to the Refund Agent...")
    return refund_agent

In [13]:
# 3. Define the Manager Agent (Triage)
triage_agent = Agent(
    name="Triage Agent",
    model="gpt-4o-mini",
    instructions="You are a front desk greeter. Answer general questions. If the user mentions refunds, use the transfer tool.",
    functions=[transfer_to_refund_agent]
)

In [14]:
# Function to run the chat
def run_swarm_chat(query, active_agent):
    print(f"\n--- User: {query} ---")

    # Run the swarm client
    response = client.run(
        agent=active_agent,
        messages=[{"role": "user", "content": query}]
    )

    # Print the response from the agent
    print(f"{response.agent.name}: {response.messages[-1]['content']}")

    # Return the new active agent (in case a handoff happened!)
    return response.agent

In [15]:
# --- Simulation ---

# Start with the Triage Agent
current_agent = triage_agent

# Turn 1: Simple greeting (Handled by Triage Agent)
current_agent = run_swarm_chat("Hi, who are you?", current_agent)

# Turn 2: Requesting a refund (Triggers Handoff to Refund Agent)
current_agent = run_swarm_chat("I want to return my shoes, item #99", current_agent)

# Turn 3: Follow up (Handled by Refund Agent, who is now in charge)
current_agent = run_swarm_chat("Thank you, that is all.", current_agent)


--- User: Hi, who are you? ---
Triage Agent: Hello! I am the front desk greeter. I'm here to assist you with any general questions you may have. How can I help you today?

--- User: I want to return my shoes, item #99 ---

[HANDOFF] Transferring user to the Refund Agent...

[MOCK BANKING SYSTEM] Processing refund for item 99...
Refund Agent: Your refund for the shoes (item #99) has been successfully processed!

--- User: Thank you, that is all. ---


KeyboardInterrupt: 